Read the Seabird CTD profiles written as CVN files.

In [99]:
import os
import glob
import logging
import datetime
import numpy as np
import matplotlib.pyplot as plt
from seabird.cnv import fCNV

In [137]:
logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [91]:
figdir = "/home/ctroupin/PycharmProjects/Stareso-Data-Processing/figures/Profiles/"
datadir = "/home/ctroupin/Data/Stareso/CTD/"
#datadir = '/home/ctroupin/Data/Stareso/CTD/201705/'
datafilelist = sorted(glob.glob(os.path.join(datadir, "*.cnv")))
logging.info("Working on {0} files".format(len(datafilelist)))

INFO:root:Working on 165 files


In [129]:
class CNVprofile(object):
    """Profile object storing the profiles obtained from cnv files
    
    """
    
    def __init__(self, filename, varlist=None, values=None):
        self.filename = filename
        logging.debug("Working on {0}".format(filename))
        if varlist==None:
            self.varlist = []
        if values==None:
            self.values = {}
            
    def read_from_file(self):
        """Read the variable names and the values from the specified file
        """
        
        self.varlist = ['scan', 'depth', 'salinity', 'temperature', 
                        'conductivity', 'density', 'flag']
        nvars = len(self.varlist)
        
        # Initialise variable dictionnary
        for i in range(0, nvars):
            self.values[self.varlist[i]] = []
        
        with open(self.filename, 'r', encoding='latin-1') as f:
            # Read first line
            line = f.readline()

            # Read until end of header (could add regex at some point)
            while not line.rstrip() == "*END*":
                line = f.readline()
            logger.debug(line)


            
            # Read the column names
            line = f.readline()
            logger.debug(line)
            columnnames = line.rsplit()
            logger.debug("Column names: {0}".format(columnnames))
            
            line = f.readline()
            while line:
                data = line.rsplit()
                for i in range(0, nvars):
                    self.values[self.varlist[i]].append(float(data[i]))
                    line = f.readline()
                    
    def add_to_plot(self, varlist, **kwargs):
        """Add the specified variables to the figure
        
        Parameters
        varlist: list of variables (strings)
        """
        for variables in varlist[1:]:
            plt.plot(self.values[varlist[0]], self.values[variables], **kwargs)
            
    @property
    def get_date(self):
        """Get the date (datetime object) corresponding to the profile
        """
        fname = os.path.basename(self.filename)
        date = datetime.datetime(int(fname[:4]), int(fname[4:6]), int(fname[6:8]))
        logging.info("Date: {0}".format(date))
        return date
        

In [131]:
P = CNVprofile(datafilelist[0])
profiledate = P.get_date

DEBUG:root:Working on /home/ctroupin/Data/Stareso/CTD/20120223_Tete_canyon_100.cnv
INFO:root:Date: 2012-02-23 00:00:00


# Make the plots

## One plot per profile

In [132]:
for datafile in datafilelist:
    P = CNVprofile(datafile)
    P.read_from_file()
    P.add_to_plot(['temperature', 'depth'])
    plt.gca().invert_yaxis()
    plt.xlabel('Temperature ($^{\circ}C$)')
    plt.ylabel('Depth (m)', rotation=0, ha='right')
    profiledate = P.get_date
    plt.title(datetime.datetime.strftime(profiledate, '%Y-%m-%d'), fontsize=20)
    figname = os.path.basename(P.filename).split('.')[0]
    plt.savefig(os.path.join(figdir, figname), bbox_inches='tight')
    #plt.show()
    plt.close()

DEBUG:root:Working on /home/ctroupin/Data/Stareso/CTD/20120223_Tete_canyon_100.cnv
DEBUG:root:*END*

DEBUG:root:Scan   Depth [m] Salinity  Temperature [deg C]  Conductivity [S/m]   Density [Kg/m3] Flag

DEBUG:root:Column names: ['Scan', 'Depth', '[m]', 'Salinity', 'Temperature', '[deg', 'C]', 'Conductivity', '[S/m]', 'Density', '[Kg/m3]', 'Flag']
INFO:root:Date: 2012-02-23 00:00:00
DEBUG:root:Working on /home/ctroupin/Data/Stareso/CTD/20120301_Tete_canyon_100.cnv
DEBUG:root:*END*

DEBUG:root:Scan   Depth [m] Salinity  Temperature [deg C]  Conductivity [S/m]   Density [Kg/m3] Flag

DEBUG:root:Column names: ['Scan', 'Depth', '[m]', 'Salinity', 'Temperature', '[deg', 'C]', 'Conductivity', '[S/m]', 'Density', '[Kg/m3]', 'Flag']
INFO:root:Date: 2012-03-01 00:00:00
DEBUG:root:Working on /home/ctroupin/Data/Stareso/CTD/20120315_Tete_canyon_100.cnv
DEBUG:root:*END*

DEBUG:root:Scan   Depth [m] Salinity  Temperature [deg C]  Conductivity [S/m]   Density [Kg/m3] Flag

DEBUG:root:Column names: ['

## All the profiles on the same plot

In [138]:
fig = plt.figure()
for datafile in datafilelist:
    P = CNVprofile(datafile)
    P.read_from_file()
    P.add_to_plot(['temperature', 'depth'])
plt.gca().invert_yaxis()
plt.xlabel('Temperature ($^{\circ}C$)')
plt.ylabel('Depth (m)', rotation=0, ha='right')
figname = 'temperature_all'
plt.savefig(os.path.join(figdir, figname), bbox_inches='tight')
#plt.show()
plt.close()

NameError: name 'datafilelistμ' is not defined

In [135]:
os.path.join(figdir, figname)

'/home/ctroupin/PycharmProjects/Stareso-Data-Processing/figures/Profiles/temperature_all'